In [ ]:
import numpy as np
from skimage import io
from scipy.ndimage import gaussian_filter
import cv2
import matplotlib.pyplot as plt
from skimage import morphology
from skimage import io
from skimage.util import img_as_ubyte
from skimage.filters import threshold_otsu
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy import ndimage as ndi
from sklearn.ensemble import IsolationForest
from skimage.draw import circle_perimeter
from skimage.io import imsave
from skimage.draw import disk
import ipywidgets as widgets
from sklearn.decomposition import FastICA
from sklearn.preprocessing import MinMaxScaler
import os
import cv2

In [ ]:
# Load the image
image = io.imread('https://raw.githubusercontent.com/sburkitt/gold_id/main/1y1_lrm_single_fov.tif')

# Define the channels and their names
channels = {0: 'DAPI', 1: 'FITC', 2: 'TRITC', 3: 'CY5'}

# Initialize a dictionary to hold the images for each channel
images = {name: image[..., channel] for channel, name in channels.items()}

# Background processing

In [ ]:
def simple_background_subtraction(image):
    background = np.median(image)
    return image - background

def high_pass_filter(image, sigma=50):
    background = gaussian_filter(image, sigma)
    return background - image

def mog2_background_subtraction(image):
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    fg_mask = bg_subtractor.apply(image)
    return fg_mask - image

def otsu_threshold(image):
    _, image_subtracted = cv2.threshold(image, 0, 4095, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return image_subtracted

def tophat_transform_white(image): 
    disk_size = morphology.disk(25)
    white_tophat = morphology.white_tophat(image, disk_size)
    return white_tophat

# image whitening 


In [ ]:
def ica_whitening(image):
    # Flatten the image
    flat_image = image.flatten()

    # Reshape the image to 2D (FastICA requires 2D inputs)
    reshaped_image = flat_image.reshape(-1, 1)
    
    # Apply FastICA
    ica = FastICA()
    whitened_image = ica.fit_transform(reshaped_image)

    # Reshape the whitened image back to the original shape
    whitened_image = whitened_image.reshape(image.shape)

    return whitened_image

def zca_whitening(image):
    # Convert the image to float type for more precise calculations
    image = image.astype(float)

    # Calculate the mean of the image
    mean = image.mean()

    # Subtract the mean from the image. This centers the image around 0
    image -= mean

    # Calculate the covariance matrix of the image
    sigma = np.dot(image.T, image) / image.size

    # Perform Singular Value Decomposition (SVD) on the covariance matrix
    # U contains the left singular vectors, S contains the singular values, and V contains the right singular vectors
    U, S, V = np.linalg.svd(sigma)

    # A small constant for numerical stability
    epsilon = 1e-5

    # Calculate the ZCA whitening matrix. This matrix, when applied to the image, will reduce the redundancy in the image's pixels
    ZCAMatrix = np.dot(np.dot(U, np.diag(1.0 / np.sqrt(S + epsilon))), U.T)

    # Apply the ZCA whitening matrix to the image and add the mean back to the result
    return np.dot(image, ZCAMatrix) + mean

def pca_whitening(image):
    # Convert the image to float
    image = image.astype(float)

    # Calculate the covariance matrix
    cov = np.cov(image)

    # Calculate the eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(cov)

    # Sort the eigenvalues and corresponding eigenvectors
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]

    # Compute the inverse square root of the eigenvalues
    sqrt_eigenvalues = np.sqrt(eigenvalues + 1e-5)

    # Compute the whitened image
    whitened_image = np.dot(eigenvectors, np.dot(np.diag(1.0/sqrt_eigenvalues), np.dot(eigenvectors.T, image)))

    # Scale the whitened image back to the range [0, 4095]
    scaler = MinMaxScaler(feature_range=(0, 4095))
    scaled_image = scaler.fit_transform(whitened_image)

    return scaled_image

# Gaussian blur
def blur(image):
    # Ensure kernel size is odd
    kernel_size=9
    if kernel_size % 2 == 0:
        kernel_size += 1
        
    # Calculate the local mean of the image
    blurimg = cv2.blur(image, (int(kernel_size), int(kernel_size)))
    return blurimg


## Selecting Preprocessing steps

In [ ]:
# Define the available preprocessing and background subtraction steps
preprocessing_steps = {
    'No Processing': lambda x: x,
    'ICA Whitening': ica_whitening,
    'ZCA Whitening': zca_whitening,
    'PCA Whitening': pca_whitening,
    'Gaussian Blur': blur
}

background_subtraction_steps = {
    'No Processing': lambda x: x,
    'Simple Background Subtraction': simple_background_subtraction,
    'High Pass Filter': high_pass_filter,
    'MOG2 Background Subtraction': mog2_background_subtraction,
    'Otsu Threshold': otsu_threshold,
    'Tophat Transform White': tophat_transform_white
}

# Create checkboxes for the user to select the preprocessing steps
checkboxes_preprocessing = widgets.SelectMultiple(
    options=preprocessing_steps.keys(),
    description='Preprocessing:',
)

# Create checkboxes for the user to select the background subtraction steps
checkboxes_background_subtraction = widgets.SelectMultiple(
    options=background_subtraction_steps.keys(),
    description='Background Subtraction:',
)

# Create a dropdown menu for the user to select the order of the steps
dropdown_order = widgets.Dropdown(
    options=['Preprocessing first', 'Background subtraction first'],
    value='Preprocessing first',
    description='Order:',
)

# Initialize the processed_images dictionary
processed_images = {}

# Define a function to apply the selected preprocessing and background subtraction steps
def apply_preprocessing_steps(b):
    selected_preprocessing_steps = checkboxes_preprocessing.value
    selected_background_subtraction_steps = checkboxes_background_subtraction.value
    order = dropdown_order.value
    for channel in ['DAPI', 'CY5']:
        processed_image = images[channel]
        # Apply the selected preprocessing and background subtraction steps in the selected order
        if order == 'Preprocessing first':
            for step in selected_preprocessing_steps:
                processed_image = preprocessing_steps[step](processed_image)
            for step in selected_background_subtraction_steps:
                processed_image = background_subtraction_steps[step](processed_image)
        else:
            for step in selected_background_subtraction_steps:
                processed_image = background_subtraction_steps[step](processed_image)
            for step in selected_preprocessing_steps:
                processed_image = preprocessing_steps[step](processed_image)
        
        # Store the processed image in the dictionary
        processed_images[channel + ' ' + ' + '.join(selected_preprocessing_steps) + ' + ' + ' + '.join(selected_background_subtraction_steps)] = processed_image

# Create a button that the user can click to start the processing
button_process = widgets.Button(description='Start Processing')
button_process.on_click(apply_preprocessing_steps)

# Define a function to save the processed images
def save_processed_images(b):
    # Create a directory to save the processed images
    os.makedirs('processed_images', exist_ok=True)
    
    # Save each processed image
    for name, image in processed_images.items():
        cv2.imwrite(f'processed_images/{name}.tif', image)

# Create a button that the user can click to save the processed images
button_save = widgets.Button(description='Save Images')
button_save.on_click(save_processed_images)

# Define a function to display the processed images
def display_processed_images(b):
    for channel in ['DAPI', 'CY5']:
        processed_image = processed_images[channel + ' ' + ' + '.join(checkboxes_preprocessing.value) + ' + ' + ' + '.join(checkboxes_background_subtraction.value)]
        # Display the original and processed images
        fig, axs = plt.subplots(1, 3, figsize=(20, 5))
        axs[0].imshow(images[channel], cmap='gray')
        axs[0].set_title(f'Original {channel} Image')
        axs[1].imshow(processed_image, cmap='gray')
        axs[1].set_title(f'First Step Processed {channel} Image')
        axs[2].imshow(processed_image, cmap='gray')
        axs[2].set_title(f'Second Step Processed {channel} Image')
        plt.show()

# Create a button that the user can click to display the processed images
button_display = widgets.Button(description='Display Images')
button_display.on_click(display_processed_images)

# Display the widgets
display(checkboxes_preprocessing, checkboxes_background_subtraction, dropdown_order, button_process, button_save, button_display)


In [ ]:
print(processed_images.keys())


# isolation forest and anomaly  

In [ ]:
# Define the size of the sub-images
sub_image_size = 16

sub_images = {channel: [] for channel in ['DAPI', 'CY5']}

for channel in ['DAPI', 'CY5']:
    key = channel + ' ' + ' + '.join(checkboxes_preprocessing.value) + ' + ' + ' + '.join(checkboxes_background_subtraction.value)
    if key in processed_images:
        image = processed_images[key]
        # Calculate the number of sub-images in each dimension
        num_sub_images_x = image.shape[1] // sub_image_size
        num_sub_images_y = image.shape[0] // sub_image_size

    # Split the image into sub-images
    for i in range(num_sub_images_y):
        for j in range(num_sub_images_x):
            sub_image = image[i*sub_image_size:(i+1)*sub_image_size, j*sub_image_size:(j+1)*sub_image_size]
            sub_images[channel].append(sub_image)


In [ ]:
# Initialize a dictionary to hold the models for each channel
models = {}

# List of channels to train on
train_channels = ['DAPI', 'CY5']

for channel in train_channels:
    # Get the sub-images for the current channel
    sub_images_channel = sub_images[channel]
    
    # Flatten the sub-images for each channel
    flattened_sub_images = [sub_image.flatten() for sub_image in sub_images_channel]

    # Train an Isolation Forest model on the flattened sub-images
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=0.5,max_features=1.0)
    model.fit(flattened_sub_images)

    # Store the model in the dictionary
    models[channel] = model

In [ ]:
#predict function will assign a value of either 1 or -1 to each sample in the data set 
#A value of 1 indicates that a point is a normal point while a value of -1 indicates that it is an anomaly.

# Predict the anomaly scores
# Get the model for the 'DAPI' channel
model_dapi = models['DAPI']

# Get the model for the 'CY5' channel
model_cy5 = models['CY5']

# Make predictions on the 'DAPI' channel
predictions_dapi = model_dapi.predict(flattened_sub_images)

# Make predictions on the 'CY5' channel
predictions_cy5 = model_cy5.predict(flattened_sub_images)

In [ ]:
#Plotting  the prediction scores 

# Convert the flattened sub-images back to 2D for each channel
sub_images_2d_dapi = [sub_image.reshape(sub_image_size, sub_image_size) for sub_image in sub_images['DAPI']]
sub_images_2d_cy5 = [sub_image.reshape(sub_image_size, sub_image_size) for sub_image in sub_images['CY5']]

# Calculate the sum of each 2D sub-image for each channel
sums_dapi = [np.sum(sub_image) for sub_image in sub_images_2d_dapi]
sums_cy5 = [np.sum(sub_image) for sub_image in sub_images_2d_cy5]

# Create a scatter plot of the sum of each sub-image vs. the prediction score for the 'DAPI' channel
plt.figure(figsize=(10, 5))
plt.scatter(sums_dapi, predictions_dapi, c=predictions_dapi, cmap='RdBu')
plt.xlabel('Sum of Sub-Image')
plt.ylabel('Prediction Score')
plt.title('Predictions for DAPI Channel')
plt.show()

# Create a scatter plot of the sum of each sub-image vs. the prediction score for the 'CY5' channel
plt.figure(figsize=(10, 5))
plt.scatter(sums_cy5, predictions_cy5, c=predictions_cy5, cmap='RdBu')
plt.xlabel('Sum of Sub-Image')
plt.ylabel('Prediction Score')
plt.title('Predictions for CY5 Channel')
plt.show()

In [ ]:
#visualizing the sub images and their prediction scores: 

# Get the sub-images for each channel that were marked as anomalies
anomaly_sub_images = {
    'DAPI': [sub_images_2d_dapi[i] for i, prediction in enumerate(predictions_dapi) if prediction == -1],
    'CY5': [sub_images_2d_cy5[i] for i, prediction in enumerate(predictions_cy5) if prediction == -1]
}

# Create a function to display a sub-image
def display_sub_image(channel, i):
    plt.imshow(anomaly_sub_images[channel][i], cmap='gray')
    plt.show()

# Create a dropdown menu to select the channel
dropdown = widgets.Dropdown(options=['DAPI', 'CY5'], value='DAPI', description='Channel:')

# Create a slider to select the sub-image index
slider = widgets.IntSlider(min=0, max=len(anomaly_sub_images[dropdown.value])-1, step=1, value=0)

# Update the maximum value of the slider when the dropdown value changes
def update_slider(*args):
    slider.max = len(anomaly_sub_images[dropdown.value])-1
dropdown.observe(update_slider, 'value')

# Create interactive widgets to display the sub-images
widgets.interact(display_sub_image, channel=dropdown, i=slider)



In [ ]:
# Initialize a dictionary to hold the anomaly scores for each channel
anomaly_scores = {channel: [] for channel in train_channels}  # Use channel names as keys

for channel_name, image in images.items():  # Iterate over channel names and images
    # Check if the current channel is one of the channels we trained a model on
    if channel_name in train_channels:
        # Calculate the number of sub-images in each dimension
        num_sub_images_x = image.shape[1] // sub_image_size
        num_sub_images_y = image.shape[0] // sub_image_size

        # Split the image into sub-images and calculate the anomaly score for each sub-image
        for i in range(num_sub_images_y):
            for j in range(num_sub_images_x):
                sub_image = image[i*sub_image_size:(i+1)*sub_image_size, j*sub_image_size:(j+1)*sub_image_size]
                anomaly_score = models[channel_name].decision_function([sub_image.flatten()])[0]  # Use channel name as key
                anomaly_scores[channel_name].append((i, j, anomaly_score))  # Use channel name as key



In [ ]:
#plotting the anomaly scores

# For each channel
for channel, anomaly_scores_channel in anomaly_scores.items():
    # Extract the anomaly scores
    scores = [score for i, j, score in anomaly_scores_channel]
    
    # Create a new figure
    plt.figure(figsize=(20, 5))
    
    # Create a histogram of the anomaly scores and get the patches
    counts, bins, patches = plt.hist(scores, bins=50)
    
    # For each patch
    for count, patch in zip(counts, patches):
        # Get the height and width of the patch
        height = patch.get_height()
        width = patch.get_width()
        
        # Calculate the position of the text
        x = patch.get_x() + width / 2
        y = height
        
        # Add the text to the plot
        plt.text(x, y, str(int(count)), ha='center', va='bottom')
    
    # Set the title and labels
    plt.title(f'Anomaly Scores for {channel} Channel')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Frequency')
    
    # Display the plot
    plt.show()



In [ ]:
# Create a function to display sub-images with anomaly scores less than a threshold
def display_anomalies(channel, threshold, index):
    # Get the anomaly scores for the selected channel
    anomaly_scores_channel = anomaly_scores[channel]

    # Filter the sub-images based on the anomaly score threshold
    filtered_sub_images = [sub_images[channel][i] for i, (i, j, score) in enumerate(anomaly_scores_channel) if score < threshold]

    # Print the count of sub-images
    print(f'Number of anomalies: {len(filtered_sub_images)}')

    # Display the sub-image at the selected index
    if filtered_sub_images:
        plt.imshow(filtered_sub_images[index], cmap='gray')
        plt.show()
    else:
        print('No anomalies found with the current threshold.')

# Create a dropdown menu to select the channel
dropdown = widgets.Dropdown(options=['DAPI', 'CY5'], value='DAPI', description='Channel:')

# Create a slider to select the anomaly score threshold
slider_threshold = widgets.FloatSlider(min=-.15, max=-.01, step=0.001, value=0.0, description='Threshold:')

# Create a slider to select the index of the sub-image to display
slider_index = widgets.IntSlider(min=0, max=100, step=1, value=0, description='Sub Image Index:')  

# Create interactive widgets to display the anomalies
widgets.interact(display_anomalies, channel=dropdown, threshold=slider_threshold, index=slider_index)


In [ ]:
# Initialize a dictionary to hold the recombined anomaly images for each channel
recombined_anomaly_images = {channel: np.zeros_like(images[channel]) for channel in channels.values()}


anomaly_thresholds = {channel: 0 for channel in channels.values()}  # Set initial thresholds to 0

#You can adjust the detection thresholds by modifying the values in the anomaly_thresholds dictionary. 
#For example, to increase the threshold for the 'DAPI' channel to -0.1, you can do:

anomaly_thresholds['DAPI'] = -0.1
anomaly_thresholds['CY5'] = -0.1

# Initialize a dictionary to hold the anomaly images for each channel
# Convert grayscale images to RGB images
anomaly_images = {channel: np.stack([np.copy(images[channel])]*3, axis=-1) for channel in channels.values()}  # Make a copy of the original images
# Initialize a dictionary to hold the recombined anomaly images for each channel
recombined_anomaly_images = {channel: np.zeros_like(images[channel]) for channel in channels.values()}


anomaly_thresholds = {channel: 0 for channel in channels.values()}  # Set initial thresholds to 0

anomaly_thresholds['DAPI'] = -0.1 #initial values 
anomaly_thresholds['CY5'] = -0.2

# Create a function to plot images with anomalies based on the thresholds
def plot_anomalies(threshold_dapi, threshold_cy5):
    # Set the thresholds
    anomaly_thresholds['DAPI'] = threshold_dapi
    anomaly_thresholds['CY5'] = threshold_cy5

    # Initialize a dictionary to hold the anomaly images for each channel
    anomaly_images = {channel: np.stack([np.copy(images[channel])]*3, axis=-1) for channel in channels.values()}  # Make a copy of the original images

    for channel, anomaly_scores_channel in anomaly_scores.items():
        for i, j, anomaly_score in anomaly_scores_channel:
            if anomaly_score < anomaly_thresholds[channel]:  # Use the threshold for the current channel
                # Draw a circle around the anomaly
                rr, cc = circle_perimeter(i*sub_image_size + sub_image_size//2, j*sub_image_size + sub_image_size//2, sub_image_size//2 + 1)
                rr -= i*sub_image_size  # Adjust the coordinates to the sub-image's coordinate system
                cc -= j*sub_image_size  # Adjust the coordinates to the sub-image's coordinate system
                # Clip the indices to the valid range
                rr = np.clip(rr, 0, sub_image_size-1)
                cc = np.clip(cc, 0, sub_image_size-1)
                sub_image = anomaly_images[channel][i*sub_image_size:(i+1)*sub_image_size, j*sub_image_size:(j+1)*sub_image_size]
                sub_image[rr, cc] = [255, 0, 0]  # Draw the circle in red on the sub-image

    # Plot the images with anomalies
    fig, axs = plt.subplots(2, 1, figsize=(30, 30))
    axs[0].imshow(anomaly_images['DAPI'], cmap='gray')
    axs[0].set_title('DAPI with anomalies')
    axs[1].imshow(anomaly_images['CY5'], cmap='gray')
    axs[1].set_title('CY5 with anomalies')
    plt.show()

# Create sliders to select the anomaly score thresholds
slider_threshold_dapi = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.1, description='Threshold DAPI:')
slider_threshold_cy5 = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.2, description='Threshold CY5:')

# Create interactive widgets to plot the anomalies
widgets.interact(plot_anomalies, threshold_dapi=slider_threshold_dapi, threshold_cy5=slider_threshold_cy5)

